In [1]:
import lhapdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt


# Global Constants
M1_test=np.sqrt(0.8)
Kp2A=0.57
Pp2A=0.12

ee=1
eU=2/3
eUbar=-2/3
eD=-1/3
eDbar=1/3
eS=-1/3
eSbar=1/3

AlphaU_test=1.0
BetaU_test=6.6
AlphaD_test=1.9
BetaD_test=10
AlphaS_test=0
BetaS_test=0

NU_test=0.18
NUbar_test=-0.01
ND_test=-0.52
NDbar_test=-0.06
NS_test=0
NSbar_test=0

# Here the data sets are listed and collected into and Array called "DataFilesArray"
Dat1='Data/HERMES_p_2009.csv'
Dat2='Data/HERMES_p_2020.csv'
Dat3='Data/COMPASS_d_2009.csv'
Dat4='Data/COMPASS_p_2015.csv'
DataFilesArray=[Dat1,Dat2,Dat3,Dat4]
PDFdataset = lhapdf.mkPDF("JAM19PDF_proton_nlo")
#PDFdataset = lhapdf.mkPDF("CT10nnlo")
FF_pion_dataset = ["JAM19FF_pion_nlo"]
FF_kaon_dataset = ["JAM19FF_kaon_nlo"]

In [2]:
def ks2Avg(m1,kperp2Avg):
    test_ks2Avg=((m1**2)*kperp2Avg)/((m1**2)+kperp2Avg)
    return test_ks2Avg

def A0(z,pht,m1,kperp2Avg,pperp2Avg,eCharg):
    tempA0part1=(((z**2)*kperp2Avg+pperp2Avg)*((ks2Avg(m1,kperp2Avg))**2))/((((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg)**2)*kperp2Avg)
    tempA0part21=(pht**2)*(z**2)*(ks2Avg(m1,kperp2Avg) - kperp2Avg)
    tempA0part22=((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg)*((z**2)*kperp2Avg+pperp2Avg)
    tempA0part2=np.exp(-tempA0part21/tempA0part22)
    tempA0part3=(np.sqrt(2*eCharg))*z*pht/m1
    tempA0=tempA0part1*tempA0part2*tempA0part3
    return tempA0

def NNq(x,Nq,aq,bq):
    tempNNq = Nq*(x**aq)*((1-x)**(bq))*((aq+bq)**(aq+bq))/((aq**aq)*(bq**bq))
    return tempNNq

def NNqbar(x,Nqbar):
    tempNNqbar = Nqbar
    return tempNNqbar

def xFxQ2(dataset,flavor,x,QQ):
    temp_parton_dist_x=(dataset.xfxQ2(flavor, x, QQ))
    return temp_parton_dist_x

def zFzQ(dataset,flavor,zz,QQ):
    # Here "0" represents the central values from the girds
    temp_zD1=lhapdf.mkPDF(dataset[0], 0)
    zD1_vec=(temp_zD1.xfxQ2(flavor,zz,QQ))
    return zD1_vec

def Asymmetry(QQ,x,z,pht,m1,Nq,aq,bq,Nqbar,eq,eqbar,lhaqID,lhaqbarID):
    kperp2Avg=Kp2A
    pperpAvg=Pp2A
    eCharg=ee
    if(lhaqID==3):
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    elif(lhaqbarID==-3):
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_kaon_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    else:
        Ucontribution1 = NNq(x,Nq,aq,bq)*(eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution1 = NNqbar(x,Nqbar)*(eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        Ucontribution2 = (eq**2)*xFxQ2(PDFdataset,lhaqID,x,QQ)*zFzQ(FF_pion_dataset,lhaqID,z,QQ)
        dbarcontribution2 = (eqbar**2)*xFxQ2(PDFdataset,lhaqbarID,x,QQ)*zFzQ(FF_pion_dataset,lhaqbarID,z,QQ)
        tempNumerator = Ucontribution1 + dbarcontribution1
        tempDenominator = Ucontribution2 + dbarcontribution2
        tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    return tempASivPiP

In [ ]:
def ASivFitHadron(KVhad, Nu, Nd, Ns, Nubar, Ndbar, Nsbar, au, ad, as0, bu, bd, bs, m1):    
    res = []
    kinspp = kinsandflag[kinsandflag[:, 4] == 0, :4]
    if(hadron=="pi+"):
        Qflag=2
        AniQflag=-1
        eQ=eU
        eQbar=eDbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Ndbar"]
    elif(hadron=="pi-"):
        Qflag=1
        AniQflag=-2
        eQ=eD
        eQbar=eUbar 
        m1= parms["m1"]
        NQ = parms["Nd"]
        alphaQ = parms["alphad"]
        betaQ = parms["betad"]
        NQbar = parms["Nubar"]
    elif(hadron=="pi0"):
        Qflag=1
        AniQflag=-1
        eQ=eU
        eQbar=eUbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Nubar"]
    elif(hadron=="k+"):
        Qflag=2
        AniQflag=-3
        eQ=eU
        eQbar=eSbar
        m1= parms["m1"]
        NQ = parms["Nu"]
        alphaQ = parms["alphau"]
        betaQ = parms["betau"]
        NQbar = parms["Nsbar"]
    elif(hadron=="k-"):
        Qflag=3
        AniQflag=-2
        eQ=eS
        eQbar=eUbar
        m1= parms["m1"]
        NQ = parms["Ns"]
        alphaQ = parms["alphas"]
        betaQ = parms["betas"]
        NQbar = parms["Nubar"]
    ################
    QQ,x,z,pht=KV
    array_size=len(x)
    tempASivHad_val=[]
    for i in range(0,array_size):
        tempASivHad=Asymmetry(QQ[i],x[i],z[i],pht[i],m1,NQ,alphaQ,betaQ,NQbar,eQ,eQbar,Qflag,AniQflag)
        tempASivHad_val.append(tempASivHad)
    return tempASivHad_val